In [1]:
def send_radial_gaia_query(query_size=1000000, distance=200, **kwargs):
    """
    Sends an archive query for d < 200 pc, with additional filters taken from
    Gaia Data Release 2: Observational Hertzsprung-Russell diagrams (Sect. 2.1)
    Gaia Collaboration, Babusiaux et al. (2018)
    (https://doi.org/10.1051/0004-6361/201832843)

    NOTE: 10000000 is a maximum query size (~76 MB / column)

    Additional keyword arguments are passed to TapPlus.launch_job_async method.
    """
    from astroquery.gaia import Gaia

#     gaia = TapPlus(url="http://gea.esac.esa.int/tap-server/tap")

    job = Gaia.launch_job_async("select top {}".format(query_size)+
                #" lum_val, teff_val,"
                #" ra, dec, parallax,"
                "designation, bp_rp, phot_g_mean_mag+5*log10(parallax)-10 as mg"
         " from gaiadr2.gaia_source"
         " where parallax_over_error > 10"
         " and visibility_periods_used > 8"
         " and phot_g_mean_flux_over_error > 50"
         " and phot_bp_mean_flux_over_error > 20"
         " and phot_rp_mean_flux_over_error > 20"
         " and phot_bp_rp_excess_factor <"
            " 1.3+0.06*power(phot_bp_mean_mag-phot_rp_mean_mag,2)"
         " and phot_bp_rp_excess_factor >"
            " 1.0+0.015*power(phot_bp_mean_mag-phot_rp_mean_mag,2)"
         " and astrometric_chi2_al/(astrometric_n_good_obs_al-5)<"
            "1.44*greatest(1,exp(-0.4*(phot_g_mean_mag-19.5)))"
         +" and 1000/parallax <= {}".format(distance), **kwargs)

    return job

In [2]:
## 200 datas

import numpy as np

try:
    gaiarec = np.recfromcsv("gaia-hrd-dr2-200pc-200data.csv")
    bp_rp, mg = gaiarec.bp_rp, gaiarec.mg
except OSError:
    job = send_radial_gaia_query(dump_to_file=True, output_format="csv",
                                 output_file="gaia-hrd-dr2-200pc-200data.csv",
                                 query_size=200)
    r = job.get_results()
    bp_rp = r['bp_rp'].data
    mg = r['mg'].data

In [32]:
## 100k datas

import numpy as np

try:
    gaiarec = np.recfromcsv("gaia-hrd-dr2-200pc.csv")
    bp_rp, mg = gaiarec.bp_rp, gaiarec.mg
except OSError:
    job = send_radial_gaia_query(dump_to_file=True, output_format="csv",
                                 output_file="gaia-hrd-dr2-200pc.csv",
                                 query_size=100000)
    r = job.get_results()
    bp_rp = r['bp_rp'].data
    mg = r['mg'].data

## Doing some absolute number stuff


In [10]:
import pandas as pd

df = pd.read_csv("gaia-hrd-dr2-200pc-200data.csv")
df

,designation,bp_rp,mg
0,Gaia DR2 4539783936566461056,0.747162,13.608096
1,Gaia DR2 2812831909590865536,0.283896,13.253650
2,Gaia DR2 4071809698800279424,1.008385,12.452089
3,Gaia DR2 6778893049604377472,0.510580,12.923529
4,Gaia DR2 793545682392321280,0.174200,12.599646
...,...,...,...
195,Gaia DR2 3905140808491432192,0.407551,12.929628
196,Gaia DR2 5003144941401554944,0.671535,13.991094
197,Gaia DR2 547189347590908160,0.493996,13.518657
198,Gaia DR2 6757553079406649088,2.755840,12.139256


In [13]:
df['bp_rp'] = df['bp_rp'].abs()
df.head(50)
df.to_csv (r'C:\Users\ianch\Desktop\Year 3 Sem 1\TDS3401 Data Visualization\Project\Data\absolute_val.csv', index = False, header=True)

# some learning i found online

In [8]:
from astroquery.gaia import Gaia

In [10]:
tables = Gaia.load_tables(only_names=True)

INFO: Retrieving tables... [astroquery.utils.tap.core]
INFO: Parsing tables... [astroquery.utils.tap.core]
INFO: Done. [astroquery.utils.tap.core]


In [11]:
for table in tables:
    print(table.name)

external.apassdr9
external.gaiadr2_astrophysical_parameters
external.gaiadr2_geometric_distance
external.gaiaedr3_distance
external.gaiaedr3_gcns_main_1
external.gaiaedr3_gcns_rejected_1
external.gaiaedr3_spurious
external.galex_ais
external.ravedr5_com
external.ravedr5_dr5
external.ravedr5_gra
external.ravedr5_on
external.sdssdr13_photoprimary
external.skymapperdr1_master
external.skymapperdr2_master
external.tmass_xsc
gaiadr1.aux_qso_icrf2_match
gaiadr1.ext_phot_zero_point
gaiadr1.allwise_best_neighbour
gaiadr1.allwise_neighbourhood
gaiadr1.gsc23_best_neighbour
gaiadr1.gsc23_neighbourhood
gaiadr1.ppmxl_best_neighbour
gaiadr1.ppmxl_neighbourhood
gaiadr1.sdss_dr9_best_neighbour
gaiadr1.sdss_dr9_neighbourhood
gaiadr1.tmass_best_neighbour
gaiadr1.tmass_neighbourhood
gaiadr1.ucac4_best_neighbour
gaiadr1.ucac4_neighbourhood
gaiadr1.urat1_best_neighbour
gaiadr1.urat1_neighbourhood
gaiadr1.cepheid
gaiadr1.phot_variable_time_series_gfov
gaiadr1.phot_variable_time_series_gfov_statistical_param

In [12]:
meta = Gaia.load_table('gaiadr2.gaia_source')

Retrieving table 'gaiadr2.gaia_source'


In [13]:
print(meta)

TAP Table name: gaiadr2.gaiadr2.gaia_source
Description: This table has an entry for every Gaia observed source as listed in the
Main Database accumulating catalogue version from which the catalogue
release has been generated. It contains the basic source parameters,
that is only final data (no epoch data) and no spectra (neither final
nor epoch).
Num. columns: 95


In [15]:
for column in meta.columns:
    print(column.name)

solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_bad_obs_al
astrometric_gof_al
astrometric_chi2_al
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_params_solved
astrometric_primary_flag
astrometric_weight_al
astrometric_pseudo_colour
astrometric_pseudo_colour_error
mean_varpi_factor_al
astrometric_matched_observations
visibility_periods_used
astrometric_sigma5d_max
frame_rotator_object_type
matched_observations
duplicated_source
phot_g_n_obs
phot_g_mean_flux
phot_g_mean_flux_error
phot_g_mean_flux_over_error
phot_g_mean_mag
phot_bp_n_obs
phot_bp_mean_flux
phot_bp_mean_flux_error
phot_bp_mean_flux_over_error
phot_bp_mean_ma

In [16]:
query1 = """SELECT 
TOP 10 
source_id, ra, dec, parallax
FROM gaiadr2.gaia_source"""

In [17]:
job = Gaia.launch_job(query1)
print(job)

<Table length=10>
   name    dtype  unit                            description                             n_bad
--------- ------- ---- ------------------------------------------------------------------ -----
source_id   int64      Unique source identifier (unique within a particular Data Release)     0
       ra float64  deg                                                    Right ascension     0
      dec float64  deg                                                        Declination     0
 parallax float64  mas                                                           Parallax     3
Jobid: None
Phase: COMPLETED
Owner: None
Output file: 1649352193223O-result.vot.gz
Results: None


In [19]:
results = job.get_results()
type(results)

astropy.table.table.Table

In [20]:
results

source_id,ra,dec,parallax
,deg,deg,mas
int64,float64,float64,float64
4650818599339213184,85.52710638111986,-72.83993794042554,-0.4127970041272453
4650810391656783104,86.4955657093574,-71.71677790439797,-0.4387554640940034
4650807466783406976,86.69123516600645,-71.84547581758729,-1.172299361501591
4650809768883233024,86.6301579735317,-71.72218747553167,0.19774549080981288
4650818637989788672,85.54810781569199,-72.83544811344974,--
4650802725139103744,87.07351536604698,-71.94636324098207,1.0700578128810871
4650808016561801088,86.46882180878941,-71.84338052432372,-0.2375256719554848
4650810735253969408,86.52002928745368,-71.67790915238713,0.017476826728849218
